# Helper tools to process data for their use in the tool

In [ ]:
from music21 import *
import predominantmelodymakam as pmm
import json
import os

## Search patterns in score
It requires a score with just the section corresponding to the recording to be used in the tool

In [ ]:
filefolder = '../../b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE'
filename = 'b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2.musicxml'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
patterns = {
    'almaya': [
        (4, 'C5B4A4G4'),
        (3, 'E4F4G4'),
        (3, 'B-4A4G4'),
        (4, 'A4G4F4E4'),
        (4, 'G4A4B4C5'),
        (3, 'B3D4C4'),
        (7, 'F4G4E4F4E4D4C4')]
}

colors = ['red', 'blue', 'green', 'fuchsia', 'aqua', 'olive', 'maroon', 'navy', 'teal', 'purple', 'lime', 'yellow']

In [ ]:
s = converter.parse(filepath)
p = s.parts[0]
nr = p.flat.notesAndRests.stream()

Search patterns for the selected tab'

In [ ]:
tab = 'almaya'

print('Patterns found:')

found_patterns = {}

position = 'above'

for pattern in patterns[tab]:
    pl = pattern[0] # pattern length
    ps = pattern[1] # pattern sequence
    color = colors[patterns[tab].index(pattern)]
    occurrences = 0
    for i in range(len(nr)-pl):
        allNotes = True
        sequence = ''
        for j in range(pl):
            n = nr[i + j]
            if n.isNote:
                sequence += n.nameWithOctave
            else:
                allNotes = False
        if allNotes and ps == sequence:
            occurrences += 1
            found_patterns[i] = (pl, position)
            for j in range(pl):
                n = nr[i + j]
                n.style.color = color
    if position == 'above':
        position = 'below'
    elif position == 'below':
        position = 'above'
            
    print('  {}: {} ({})'.format(ps, occurrences, color))

# s.show()

Add bracket lines for the found patterns

In [ ]:
for k in sorted(found_patterns.keys()):
    line = spanner.Slur()
    for i in range(found_patterns[k][0]):
        n = nr[k + i]
        line.addSpannedElements(n)
    line.placement = found_patterns[k][1]
    p.insert(line)

s.show()

## Beats
Transform SV annotation layer to json object

In [ ]:
filefolder = '../../b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE'
filename = 'b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE-beats.txt'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
with open(filepath, 'r') as f:
    data = f.readlines()

In [ ]:
tak = []
dum = []

for d in data:
    time = round(float(d.split('\t')[0]), 2)
    beat = d.rstrip()[-1]
    if beat == '3':
        dum.append(time)
    else:
        tak.append(time)

len(data) == len(tak) + len(dum)

In [ ]:
print(json.dumps({'tak':tak,'dum':dum}, indent=2))

## Patterns
Transform SV annotation layer to json object

In [ ]:
filefolder = '../../b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE'
filename = 'b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE-patterns.txt'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
with open(filepath, 'r') as f:
    data = f.readlines()

In [ ]:
patterns = {}

for d in data:
    start = round(float(d.split('\t')[0]), 2)
    end = round(float(d.split('\t')[0]) + float(d.split('\t')[2]), 2)
    pattern = d.split('\t')[-1].rstrip()
    if pattern[0] != 'P':
        print('Annotation error at', start)
    if pattern not in patterns:
        patterns[pattern] = [{'start':start, 'end':end}]
    else:
        patterns[pattern].append({'start':start, 'end':end})

In [ ]:
print(json.dumps(patterns, indent=2))

In [ ]:
x = 1067
print((x*9)/16)

## Extract pitch track

In [ ]:
filefolder = '../../../b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE'
filename = 'b11237b9-d45b-4b3a-a97b-ab7d198f927f-mu2-NOFADE.mp3'

filepath = os.path.join(filefolder, filename)
print(filepath)

In [ ]:
# Parameters: min_frequency, max_frequency, filter_pitch (bool), peak_distribution_threshold
extractor = pmm.PredominantMelodyMakam(min_frequency=100, max_frequency=340, filter_pitch=False, peak_distribution_threshold=2)
pitchTrack = extractor.run(filepath)

In [ ]:
pitchTrackTxt = ''
for i in pitchTrack['pitch']:
    if i[1] == 0:
        p = '0'
    else:
        p = '{:.6f}'.format(i[1])
    pitchTrackTxt += '{:.6f}\t{}\n'.format(i[0], p)

In [ ]:
minf0 = pitchTrack['settings']['minFrequency']
maxf0 = pitchTrack['settings']['maxFrequency']
pdt = pitchTrack['settings']['peakDistributionThreshold']
fp = pitchTrack['settings']['filterPitch']
filePath2write = filepath[:-4] + 'pmm{}-{}-{}-{}.txt'.format(minf0, maxf0, pdt, fp)
with open(filePath2write, 'w') as f:
    f.write(pitchTrackTxt.rstrip())